In [12]:
# | default_exp auth

In [13]:
# | hide
from nbdev.showdoc import *

In [14]:
# | exporti
import urllib.parse as parse
from dataclasses import dataclass, field
import base64
import httpx

import jiralibrary.client as gd

In [15]:
# | exporti
@dataclass
class _JiraAuth_Required:
    instance: str  # 'your-domain.atlassian.net'


@dataclass
class _JiraAuth_Optional:
    base_url: str = None

In [16]:
# | export


@dataclass
class JiraAuth(_JiraAuth_Optional, _JiraAuth_Required):
    
    pass

In [17]:
import os

jira_instance = os.environ["JIRA_INSTANCE"]

JiraAuth(instance=jira_instance)

JiraAuth(instance='onyxreporting.atlassian.net', base_url=None)

In [18]:
# | exporti
@dataclass
class _JiraAuthPat_Required:
    email: str
    pat: str = field(repr=False)

In [19]:
# | export
@dataclass
class JiraAuthPat(_JiraAuth_Optional, _JiraAuthPat_Required, _JiraAuth_Required):
    def __post_init__(self):
        if not self.instance.startswith("https://"):
            instance = f"https://{self.instance}"

        self.instance = parse.urlparse(instance).netloc
        self.base_url = instance
        
    def generate_auth_header(self):
        auth_str = base64.b64encode(f"{self.email}:{self.pat}".encode("ascii")).decode(
            "ascii"
        )
        return {"Authorization": f"Basic {auth_str}"}

    async def who_am_i(
        self, session: httpx.AsyncClient = None, debug_api: bool = False
    ):
        url = f"{self.base_url}/rest/api/2/myself"

        response = await gd.get_data(
            method="GET",
            url=url,
            auth=self,
            debug_api=debug_api,
            session=session,
        )
        return response

In [20]:
import os

jira_pat = os.environ["JIRA_PAT"]
jira_instance = os.environ["JIRA_INSTANCE"]

jira_auth = JiraAuthPat(
    pat=jira_pat, email="jae@onyxreporting.com", instance=jira_instance
)

res = await jira_auth.who_am_i(debug_api=True)

res.response

{'body': None,
 'headers': {'Accept': 'application/json',
             'Authorization': 'Basic '
                              'amFlQG9ueXhyZXBvcnRpbmcuY29tOkFUQVRUM3hGZkdGMGlUZkF4WW1fdXlYcXZfUjEyZk5JOEFwc3JsT19tQ3RiOFVWVXVaaEhZSVh3NU5CQnBBZTZwWUZacVBIQTlUbWx5MWw4QjZZMldJWkJqVzZDUFBETE1zLXU0RGFJVkNrbk5zbUdidnFScGdINUdjSWRGUkhqN1RaUWNEUGlneTY1c2FyVHZiSkQ5Y3J0WlY2ajZlZENGRlZzRUdSdDRER1ZacjFqS09CUEwwWT1COUY5RUZGOQ=='},
 'params': None,
 'url': 'https://onyxreporting.atlassian.net/rest/api/2/myself'}


{'self': 'https://onyxreporting.atlassian.net/rest/api/2/user?accountId=70121:233b7627-506e-41d8-af0c-336541c67dab',
 'accountId': '70121:233b7627-506e-41d8-af0c-336541c67dab',
 'accountType': 'atlassian',
 'emailAddress': 'jae@onyxreporting.com',
 'avatarUrls': {'48x48': 'https://secure.gravatar.com/avatar/2314f91c29ace4d7d9426d26448391f4?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public.atl-paas.net%2Finitials%2FJW-0.png',
  '24x24': 'https://secure.gravatar.com/avatar/2314f91c29ace4d7d9426d26448391f4?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public.atl-paas.net%2Finitials%2FJW-0.png',
  '16x16': 'https://secure.gravatar.com/avatar/2314f91c29ace4d7d9426d26448391f4?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public.atl-paas.net%2Finitials%2FJW-0.png',
  '32x32': 'https://secure.gravatar.com/avatar/2314f91c29ace4d7d9426d26448391f4?d=https%3A%2F%2Favatar-management--avatars.us-west-2.prod.public.atl-paas.net%2Finitials%2FJW-0.png'},
 'display

In [21]:
# | hide
import nbdev

nbdev.nbdev_export()